In [129]:
import numpy as np
import matplotlib.pyplot as plt

In [339]:
class ACO:
    def __init__(self, num_pop, graph):
        n = len(graph)

        self.population = [[0]] * num_pop # Each ant starts at city number 0
        self.dist = np.zeros(shape=(num_pop,)) # dist[ant] = distance made so far

        self.pheromon = np.random.normal(size=(n, n)) ** 2 # Some initialization, really

        self.adjacency_matrix = np.zeros(shape=(n, n), dtype=np.float32) # [i, j] = Euclidean distance between city i and j

        # Setting up the distances
        for i in range(n):
            self.adjacency_matrix[i] = np.apply_along_axis(np.linalg.norm, 1, graph[i] - graph)

        # Banning paths from city to itself 
        self.adjacency_matrix = np.where(self.adjacency_matrix == 0, np.inf, self.adjacency_matrix)

    def main_loop(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromon) # To perform pheromon decay after loop ends
            
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromon[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Setting visited cities probabilities to zero
                    for j in range(n):
                        if j in self.population[ant]:
                            tmp[j] = 0

                    # Probabilities itself
                    prob = tmp / tmp.sum() # nan's since it counts current city for wich mu = inf

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())

                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromon[current_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                    if len(self.population[ant]) == n:
                        # Connecting the last city from the first
                        self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                # There's some bug which I don't know how fix
                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromon = (1 - rho) * self.pheromon + delta_tau

In [340]:
def create_circle(n, radius=1):
    circle = []
    theta = 0
    delta = 2 * np.pi / n

    side_length = 2 * radius * np.sin(delta / 2)

    for _ in range(n):
        x = np.cos(theta)
        y = np.sin(theta)
        circle.append(np.array([x, y]))
        theta += delta


    return np.array(circle), n * side_length

In [350]:
"""
Min distance: 5.878
Min route [0 4 3 2 1]
Real ans 5.878
"""
graph, perim = create_circle(20, 1)
test = ACO(100, graph)
test.main_loop(num_iter=10_000, alpha=1, beta=0.5, rho=0, Q=6)
print("Real ans %.3f" % perim)

Iteration: 1
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 2
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 3
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 4
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 5
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 6
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 7
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 8
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 9
Min distance: 19.216
Min route [ 0 12  7  4  2 18 17 19 16 14 10  9  8  6 15 11 13  1  3  5]
Iteration: 10
Min distance: 19.216
Min route [

In [230]:
arr = np.array([0.1, 0, 0, 0.4, 0, 0, 0.5])
np.cumsum(arr)

array([0.1, 0.1, 0.1, 0.5, 0.5, 0.5, 1. ])

In [175]:
((1 - 0.30901699) ** 2 + 0.95105625 ** 2) ** .5

1.1755702917191821